In [23]:
import httplib2
import argparse
import pandas as pd
from pprint import pprint
from oauth2client.file import Storage
from oauth2client import tools
from oauth2client import client
from googleapiclient import discovery

In [17]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
CLIENT_SECRET_FILE = 'client_secret_grants.json' 
APPLICATION_NAME = 'Google Sheets API Report'
credential_path = 'sheets.googleapis.com-report.json'

In [18]:
store = Storage(credential_path)
credentials = store.get()
parser = argparse.ArgumentParser(formatter_class=argparse.RawDescriptionHelpFormatter, parents=[tools.argparser])
flags = parser.parse_args([])
if not credentials or credentials.invalid:
    flow = client.flow_from_clientsecrets(CLIENT_SECRET_FILE, SCOPES)
    flow.user_agent = APPLICATION_NAME
    if flags:
        credentials = tools.run_flow(flow, store, flags)
print('Storing credentials to ' + credential_path)

Storing credentials to sheets.googleapis.com-report.json


In [19]:
http = credentials.authorize(httplib2.Http())
discoveryUrl = ('https://sheets.googleapis.com/$discovery/rest?version=v4')
service = discovery.build('sheets', 'v4', http = http, discoveryServiceUrl = discoveryUrl)
spreadsheetId = '1CmQqTVuzqDs5WX7fNAlB43AvTsoOfYqIE266impvCJI'

In [20]:
rangeName = 'partners dict!A1:C10'

In [21]:
result = service.spreadsheets().values().get(spreadsheetId = spreadsheetId, range = rangeName).execute()
values = result.get('values', [])

In [22]:
pprint(values)

[['Партнер', 'utm_source', 'utm_medium'],
 ['СМИ2', 'smi2', 'cpm'],
 ['РБК', 'rbc', 'cpc'],
 ['Infox', 'infox', 'cpc'],
 ['Infox', 'infox-action', 'cpa']]


In [29]:
headers = values[0]
rows = values[1:]
data_top = pd.DataFrame.from_records(rows, columns = headers)
data_bottom = pd.read_csv('data/metrika_data.txt', sep = '\t')

In [33]:
data_top.head()

,Партнер,utm_source,utm_medium
0,СМИ2,smi2,cpm
1,РБК,rbc,cpc
2,Infox,infox,cpc
3,Infox,infox-action,cpa


In [42]:
data_bottom.head()

,utm_source,utm_medium,sales
0,infox,cpc,1534
1,smi2,cpm,1495
2,infox-action,cpa,735
3,rbc,cpc,59


In [50]:
result = pd.merge(data_top, data_bottom, how='outer', on=['utm_source', 'utm_medium'])

In [51]:
result.head()

,Партнер,utm_source,utm_medium,sales
0,СМИ2,smi2,cpm,1495
1,РБК,rbc,cpc,59
2,Infox,infox,cpc,1534
3,Infox,infox-action,cpa,735
